In [4]:
# Import required libraries
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv(r'D:\archive (1)\Sample - Superstore.csv', encoding='ISO-8859-1')

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a layout for the dashboard
app.layout = html.Div(style={'backgroundColor': '#f9f9f9', 'padding': '20px'}, children=[
    html.H1("Superstore Sales Dashboard", style={'textAlign': 'center', 'color': '#333'}),
    
    # Dropdown for selecting category
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': category, 'value': category} for category in df['Category'].unique()],
        value=df['Category'].unique()[0],  # Default value
        clearable=False,
        style={'width': '50%', 'margin': 'auto', 'fontSize': 18}
    ),
    
    # Graphs layout
    html.Div(style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'space-around'}, children=[
        # Sales by Sub-Category
        dcc.Graph(id='sales-by-category', style={'flex': '1 1 45%', 'margin': '10px'}),
        
        # Profit by Sub-Category
        dcc.Graph(id='profit-by-category', style={'flex': '1 1 45%', 'margin': '10px'}),
        
        # Sales Trend Over Time
        dcc.Graph(id='sales-trend', style={'flex': '1 1 100%', 'margin': '10px'}),
        
        # Sales Distribution by Region
        dcc.Graph(id='sales-by-region', style={'flex': '1 1 45%', 'margin': '10px'}),
        
        # Average Discount by Category
        dcc.Graph(id='avg-discount-by-category', style={'flex': '1 1 45%', 'margin': '10px'}),
    ])
])

# Callback to update graphs based on selected category
@app.callback(
    [dash.dependencies.Output('sales-by-category', 'figure'),
     dash.dependencies.Output('profit-by-category', 'figure'),
     dash.dependencies.Output('sales-trend', 'figure'),
     dash.dependencies.Output('sales-by-region', 'figure'),
     dash.dependencies.Output('avg-discount-by-category', 'figure')],
    [dash.dependencies.Input('category-dropdown', 'value')]
)
def update_graphs(selected_category):
    # Filter data based on selected category
    filtered_df = df[df['Category'] == selected_category]
    
    # Sales by Sub-Category
    sales_fig = px.bar(filtered_df, x='Sub-Category', y='Sales', 
                       title='Sales by Sub-Category', 
                       color='Sub-Category', 
                       color_discrete_sequence=px.colors.qualitative.Plotly)
    
    # Profit by Sub-Category
    profit_fig = px.bar(filtered_df, x='Sub-Category', y='Profit', 
                        title='Profit by Sub-Category', 
                        color='Sub-Category', 
                        color_discrete_sequence=px.colors.qualitative.Plotly)
    
    # Sales trend over time
    sales_trend = filtered_df.groupby('Order Date')['Sales'].sum().reset_index()
    sales_trend_fig = px.line(sales_trend, x='Order Date', y='Sales', 
                               title='Sales Trend Over Time', 
                               markers=True)
    
    # Sales Distribution by Region
    region_sales = df.groupby('Region')['Sales'].sum().reset_index()
    sales_by_region_fig = px.pie(region_sales, values='Sales', names='Region', 
                                  title='Sales Distribution by Region', 
                                  color_discrete_sequence=px.colors.sequential.RdBu)
    
    # Average Discount by Category
    avg_discount = df.groupby('Category')['Discount'].mean().reset_index()
    avg_discount_fig = px.bar(avg_discount, x='Category', y='Discount', 
                               title='Average Discount by Category', 
                               color='Discount', 
                               color_continuous_scale=px.colors.sequential.Viridis)
    
    return sales_fig, profit_fig, sales_trend_fig, sales_by_region_fig, avg_discount_fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
